Define:
* ATP: avg team points
* EATP: estimated ATP

Ideally for the initial value stat we want the following final data structure:
* race
* year
* stage
* rider
* team
* result
* points (UCI or PCS)

I think I can just download these results from the web (2010 to 2020) to get started - later I can use the PCS API.

I think that's it, for the APM equivalent. Reg is something like:

ATP ~ rider matrix + team FE (account for bad management/coaching)

and then take individual coefs? Need to think about absolute points vs relative points, per race etc. Propose:
ratio of points to average points per team
or: team_points/(sum(team_points)/count(teams)) 

Then will need to think about weighting later for race importance - but this gets to what we really want which is - this rider is in a race, how much do they help you win/do well

Then we want to find some way to estimate points, given stats on a course and rider. For that we want:
* race_profile - or some combination of variables to contain this data, eg
    * profile (flat, hilly, mountain)
    * length
    * temperature?
    * final 5k profile?
    * cobbles? etc
* race type (RR, TT, TTT etc)
* rider height
* rider weight
* rider age
* rider type (puncheur, rouleur etc)

Define:
* ATP: avg team points
* EATP: estimated ATP

Does it work to do something like 
ATP ~ result + points on offer + etc? it will just be dominated by those two. In EPM, points is an explainer - so maybe? But really we don't want to compare top riders to top riders as much as domestiques to domestiques - maybe? I guess comparing riders along the lines of who they beat is interesting

Then really what we want is extra data on:
* Who pulled when
* Crashes
* Attacks/time off the front

I can try to use GPT to pull these - feed in website text from PCS, eurosport etc tickers. Will take some prompt work

Could start just on TTs - TT rating, use it to work out kinks e.g. how to decay over time well


Questions
* How does raptor work with using riders' previous years values + somehow using bayes for young players?
* How many points a rider's team earns will be hugely influenced by what race they do & how many points are on offer. But bad riders will not get to go to good races. How do I deal with this?

# ELO
First project idea:
* Construct ELO for flat TTs & hilly TTs
* Play with parameters - at least think about having variable shifts based on time
* Construct evaluation metric - how well it predicts pairs of riders in a race
* Pull data for far back ish
* Use eval metric to tighten up hyperparams. Should these shift over time?
* graph from far back
* Write blog post


In [15]:
!pip install git+https://github.com/djcunningham0/multielo.git

  Cloning https://github.com/djcunningham0/multielo.git to /private/var/folders/jt/3g8y_ysd71z9ymzmxjypy9yw0000gn/T/pip-req-build-e6r6jc__
  Running command git clone --filter=blob:none --quiet https://github.com/djcunningham0/multielo.git /private/var/folders/jt/3g8y_ysd71z9ymzmxjypy9yw0000gn/T/pip-req-build-e6r6jc__
  Resolved https://github.com/djcunningham0/multielo.git to commit 440f7922b90ff87009f8283d6491eb0f704e6624
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for multielo: filename=multielo-0.4.1-py3-none-any.whl size=15752 sha256=17dfd3440643c5f99ae33f027858fc7010be561568c22f934098fc93c7c50dd6
  Stored in directory: /private/var/folders/jt/3g8y_ysd71z9ymzmxjypy9yw0000gn/T/pip-ephem-wheel-cache-6xus6si0/wheels/f7/62/40/e50107c862bdf9c98222b8af6987fdfc8a1c8bb0550a04c391
Successfully built multielo


In [2]:
from multielo import Tracker
import numpy as np
import pandas as pd
import seaborn as sns

We can use the tracker object from multielo. It expects:
* rows of events
* Columns: date, then 1st 2nd 3rd and so forth 

In [3]:
pcs_df = pd.read_csv("pcs_2023_df.csv")

In [9]:
pcs_df.head()

,race_name,race_category,uci_tour,stage_url,is_one_day_race,distance,stage_type,winning_attack_length,date,won_how,...,rider_name,rider_url,team_name,team_url,rank,status,age,time,pcs_points,uci_points
0,National Championships India ME - ITT,Men Elite,Asia Tour,race/nc-india-itt/2023/result,True,40.0,ITT,NaN,2023-01-07,NaN,...,JOHN Naveen,rider/naveen-john,NaN,NaN,1.0,DF,36.0,0:51:21,15,25.0
21,National Championships India ME - ITT,Men Elite,Asia Tour,race/nc-india-itt/2023/result,True,40.0,ITT,NaN,2023-01-07,NaN,...,RAHUL Rahul,rider/rahul-rahul,NaN,NaN,22.0,DF,NaN,0:57:59,0,0.0
1,National Championships India ME - ITT,Men Elite,Asia Tour,race/nc-india-itt/2023/result,True,40.0,ITT,NaN,2023-01-07,NaN,...,SINGH Vishavjeet,rider/vishavjeet-singh,NaN,NaN,2.0,DF,20.0,0:51:44,10,15.0
2,National Championships India ME - ITT,Men Elite,Asia Tour,race/nc-india-itt/2023/result,True,40.0,ITT,NaN,2023-01-07,NaN,...,KUMAR Dinesh,rider/dinesh-kumar,NaN,NaN,3.0,DF,27.0,0:53:14,7,10.0
3,National Championships India ME - ITT,Men Elite,Asia Tour,race/nc-india-itt/2023/result,True,40.0,ITT,NaN,2023-01-07,NaN,...,KUMAR Satish,rider/satish-kumar,NaN,NaN,4.0,DF,NaN,0:53:21,4,5.0


In [10]:
tt_df = pcs_df[pcs_df["stage_type"] == "ITT"]

In [59]:
# Initialize ELO ratings
initial_elo = 1500
k_factor = 32  # You can adjust this depending on how quickly you want ELOs to change
elo_dict = {rider: initial_elo for rider in pcs_df['rider_name'].unique()}

# Store ELOs after each race
elo_history = []

# Function to calculate expected score (probability) from ELO ratings
def expected_score(rating_a, rating_b):
    return 1 / (1 + 10 ** ((rating_b - rating_a) / 400))

# Iterate over each race
for race_date, race_group in pcs_df.groupby(['date', 'race_name']):
    # Get the riders and their ranks
    riders = race_group['rider_name'].values
    ranks = race_group['rank'].values

    # Compute new ELOs based on race results
    updated_elo_dict = elo_dict.copy()
    for i in range(len(riders)):
        for j in range(i + 1, len(riders)):
            rider_a = riders[i]
            rider_b = riders[j]
            rank_a = ranks[i]
            rank_b = ranks[j]
            
            # Expected score for rider A
            prob_a_wins = expected_score(elo_dict[rider_a], elo_dict[rider_b])
            
            # Determine the outcome (1 if A wins, 0 if B wins)
            outcome_a = 1 if rank_a < rank_b else 0
            outcome_b = 1 - outcome_a
            
            # Update ELOs
            updated_elo_dict[rider_a] += k_factor * (outcome_a - prob_a_wins)
            updated_elo_dict[rider_b] += k_factor * (outcome_b - (1 - prob_a_wins))
    
    # Record the updated ELOs for the current race date
    for rider in riders:
        elo_history.append({
            'date': race_date[0],
            'race': race_date[1],
            'rider': rider,
            'elo_rating': updated_elo_dict[rider]
        })

    # Update the global ELO dictionary with the new ratings after the race
    elo_dict.update(updated_elo_dict)

# Convert the ELO history into a DataFrame
elo_df = pd.DataFrame(elo_history)

In [ ]:
sns.relplot(data=elo_df, x='date', y='elo_rating', hue='rider_name', kind='line', aspect=2)

In [15]:
elo_df.sort_values(by='elo_rating', ascending=False).head(00)

,date,rider_name,elo_rating
66701,2023-06-11,KÜNG Stefan,2152.523469
67677,2023-06-12,KÜNG Stefan,2150.370946
68653,2023-06-13,KÜNG Stefan,2148.220575
69629,2023-06-14,KÜNG Stefan,2146.072354
70614,2023-06-15,KÜNG Stefan,2143.926282
...,...,...,...
43920,2023-05-14,GEOGHEGAN HART Tao,2073.044783
238585,2023-09-18,GANNA Filippo,2072.885843
58241,2023-06-01,ROGLIČ Primož,2072.546044
58405,2023-06-01,THOMAS Geraint,2072.240885


In [20]:
tt_df[(tt_df['date'] == '2023-06-11') & (tt_df['rider_name'] == 'KÜNG Stefan')]

,race_name,race_category,uci_tour,stage_url,is_one_day_race,distance,stage_type,winning_attack_length,date,won_how,...,rider_name,rider_url,team_name,team_url,rank,status,age,time,pcs_points,uci_points
25533,ZLM Tour,Men Elite,UCI ProSeries,race/zlm-tour/2023/stage-4,False,12.7,ITT,NaN,2023-06-11,NaN,...,KÜNG Stefan,rider/stefan-kung,Groupama - FDJ,team/groupama-fdj-2023,1.0,DF,29.0,13.31,50,60.0


In [36]:
## Measure the accuracy of the ELO model

from sklearn.metrics import log_loss, brier_score_loss

# Function to calculate expected score (probability) from ELO ratings
def expected_score(rating_a, rating_b):
    return 1 / (1 + 10 ** ((rating_b - rating_a) / 400))

# Ensure the elo_df is sorted by date and rider
elo_df = elo_df.sort_values(by=['rider_name', 'date'])

# Step 1: Precompute the latest ELO ratings before each race date for each rider
# Create a lookup table for each rider's ELO at the time of the race
elo_lookup = {}

for rider in pcs_df['rider_name'].unique():
    rider_elo_history = elo_df[elo_df['rider_name'] == rider]
    if not rider_elo_history.empty:
        # Use forward fill to carry the last known ELO rating forward up to each race date
        rider_elo_history = rider_elo_history.set_index('date').resample('D').ffill().reset_index()
        elo_lookup[rider] = rider_elo_history.set_index('date')['elo_rating']

In [57]:
# Initialize lists to store predictions and actual outcomes
pairwise_predictions = []
actual_outcomes = []

# Evaluate accuracy over the entire dataset
for race, group in tt_df.groupby('race_name'):
    race_date = group['date'].iloc[0]
    riders = group['rider_name'].values
    ranks = group['rank'].values

    # Fetch the ELO ratings for each rider at the time of the race from the lookup
    race_elos = {rider: elo_lookup[rider].loc[race_date] if rider in elo_lookup and race_date in elo_lookup[rider] else initial_elo for rider in riders}
    
    # Pairwise comparisons within the race
    for i in range(len(riders)):
        for j in range(i + 1, len(riders)):
            rider_a = riders[i]
            rider_b = riders[j]
            rank_a = ranks[i]
            rank_b = ranks[j]
            
            # Calculate expected score (probability) based on the ELO rating
            rating_a = race_elos[rider_a]
            rating_b = race_elos[rider_b]
            prob_a_wins = expected_score(rating_a, rating_b)
            prob_b_wins = 1 - prob_a_wins

            # Store the predicted probability that A beats B
            pairwise_predictions.append(prob_a_wins)
            
            # Store the actual outcome (Whether A beat B)
            actual_outcomes.append(rank_a < rank_b)

            # Debugging print statements
            #print(f"Race: {race}, Date: {race_date}, Riders: {rider_a} vs {rider_b}")
            #print(f"ELOs: {rating_a} vs {rating_b}, Prob_A_wins: {prob_a_wins}")
            #print(f"Ranks: {rank_a} vs {rank_b}, Expected: {'A wins' if rank_a < rank_b else 'B wins'}")
        

# Calculate accuracy as the percentage of correct predictions
accuracy = np.mean(np.equal([1 if pred >= 0.5 else 0 for pred in pairwise_predictions], actual_outcomes))

# Calculate log-loss (cross-entropy loss)
logloss = log_loss(actual_outcomes, pairwise_predictions)

# Calculate Brier score
brier = brier_score_loss(actual_outcomes, pairwise_predictions)

print(f'Accuracy: {accuracy * 100:.9f}%')
print(f'Log-Loss: {logloss:.4f}')
print(f'Brier Score: {brier:.4f}')

Accuracy: 88.910778602%
Log-Loss: 0.3350
Brier Score: 0.0958


In [47]:
#np.sum([0 if pred >= 0.5 else 1 for pred in pairwise_predictions] == actual_outcomes)
np.equal([0 if pred >= 0.5 else 1 for pred in pairwise_predictions], actual_outcomes)

array([ True,  True,  True, ..., False, False, False])